In [ ]:
import torch
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [ ]:
df = pd.read_csv('C:/Users/minkyu/Desktop/dacon accident prevention/진짜 시작/aug_train.csv', encoding='utf-8-sig') 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
embedding_model = SentenceTransformer('jhgan/ko-sbert-sts', device=device, use_auth_token=False)
print("embedding 모델 불러오기 성공")
 
plans = [str(x) for x in df['재발방지대책 및 향후조치계획'].tolist()] 
preds = [str(x) for x in df['pred'].tolist()] # 예측 문장이 들어있는 컬럼인지 확인

embeddings_plans = embedding_model.encode(plans)
embeddings_preds = embedding_model.encode(preds)

# 코사인 유사도 계산 함수
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

# 각 행에 대해 코사인 유사도 계산
cosine_similarity_scores = [
    cosine_similarity(plan_emb, pred_emb)
    for plan_emb, pred_emb in tqdm(zip(embeddings_plans, embeddings_preds), total=len(embeddings_plans), desc="SentenceTransformer 유사도")
]


# --- 결과를 데이터프레임에 저장 ---
df['cosine_similarity'] = cosine_similarity_scores
print('평균 코사인 유사도',df['cosine_similarity'].mean())

In [ ]:
grouped_similarity = df.groupby('공종(중분류)')['cosine_similarity'].mean()
print(grouped_similarity)